In [ ]:
import sys
from pathlib import Path
import pandas as pd

sys.path.append("../../")
from lib.stats import demographic_characteristics

### Input

In [ ]:
# Specify input paths from ARIC_NP server
path_derive54: Path = Path(
    "Z:/DATA_NP/Visits/Visit 5/derive54_np.sas7bdat"
).resolve()  # Visit 5 on ARIC_NP server
path_derive13: Path = Path(
    "Z:/DATA_NP/Visits/Visit 1/derive13_np.sas7bdat"
).resolve()  # Visit 1 on ARIC_NP server
path_ncs51: Path = Path(
    "Z:/DATA_NP/Visits/Visit 5/derive_ncs51_np.sas7bdat"
).resolve()  # Visit 5 on ARIC_NP server
path_mri: Path = Path(
    "Z:/DATA_NP/Visits/MultiVisit/V5_V11 Longitudinal MRI data/v5_v11_mri_derv_np_240221.sas7bdat"
).resolve()  # MultiVisit on ARIC_NP server

In [ ]:
# Specify input paths
path_population: Path = Path(
    "../../../data/original/aric/sample_selection/all_eleigible_samples_AS2021_25v3.xlsx"
).resolve()
path_pilot: Path = Path(
    "../../../data/original/aric/ARIC_Pilot_Updated_06032022.csv"
).resolve()
path_lipoprotein: Path = Path(
    "../../../data/original/aric/lipoproteins_6_29_23.csv"
).resolve()
path_dictionary: Path = Path("../../../data/processed/aric/dictionary.csv").resolve()

In [ ]:
# Specify output paths
path_output_lipoprotein_list: Path = Path(
    "../../../data/processed/aric/lipoprotein_list.csv"
).resolve()
path_output_pilot: Path = Path("../../../data/processed/aric/pilot.csv").resolve()
path_output_demographic_characteristics: Path = Path(
    "../../../assets/tables/aric/demographic_characteristics.csv"
).resolve()

In [ ]:
# Read files from ARIC_NP server
derive54: pd.DataFrame = pd.read_sas(
    path_derive54, format="sas7bdat", encoding="latin-1"
).convert_dtypes()
derive13: pd.DataFrame = pd.read_sas(
    path_derive13, format="sas7bdat", encoding="latin-1"
).convert_dtypes()
ncs51: pd.DataFrame = pd.read_sas(
    path_ncs51, format="sas7bdat", encoding="latin-1"
).convert_dtypes()
mri: pd.DataFrame = pd.read_sas(
    path_mri, format="sas7bdat", encoding="latin-1"
).convert_dtypes()

In [ ]:
# Read files
population: pd.DataFrame = (
    pd.read_excel(path_population, sheet_name=0, header=0)
    .rename(columns={"subjectid": "SubjectID"})
    .convert_dtypes()
)
pilot: pd.DataFrame = pd.read_csv(path_pilot)
lipoprotein: pd.DataFrame = pd.read_csv(path_lipoprotein).convert_dtypes()
dictionary: pd.DataFrame = (
    pd.read_csv(path_dictionary)
    .dropna(subset=["derive54_np", "derive13_np", "derive_ncs51_np"], how="all")
    .convert_dtypes()
)

### Define inclusion

In [ ]:
# Inclusion criteria
# 1. have fasting unthawed plasma samples at V5
population["criteria1"] = (population["criteria1"] == 1).astype(bool)

# 2. were adjudicated CN or MCI at V5
population["COGDIAG51"] = pd.Categorical(
    population["COGDIAG51"], categories=["N", "M", "D", "U"], ordered=True
)
population["cn_or_mci"] = (population["COGDIAG51"].isin(["N", "M"])).astype(bool)

# 3. have brain MRI scans at V5
ncs51["have_mri"] = ncs51["ADSIGREGVOL51"].notna()

# 4. have amyloid PET data from ARIC-PET study
population["criteria3"] = (population["criteria3"] == 1).astype(bool)

In [8]:
# Create sets of SUBJECTIDs for each criteria
c1: set[str] = set(population["SubjectID"].loc[population["criteria1"]])
c2: set[str] = set(population["SubjectID"].loc[population["cn_or_mci"]])
c3: set[str] = set(ncs51["SUBJECTID"].loc[ncs51["have_mri"]])
c4: set[str] = set(population["SubjectID"].loc[population["criteria3"]])

In [9]:
# Compute included and excluded population using set operations
id_eligible: set[str] = c1.intersection(c2, c3, c4)
id_included: set[str] = set(lipoprotein["ID"])
id_excluded: set[str] = id_eligible.difference(id_included)

### Collect data from ARIC server for excluded population

In [ ]:
# Define dictionaries that map variable names to their original names on the ARIC server
dict_derive54: dict[str, str] = (
    dictionary.dropna(subset=["derive54_np"])
    .set_index("variable_name")["derive54_np"]
    .to_dict()
)
dict_derive13: dict[str, str] = (
    dictionary.dropna(subset=["derive13_np"])
    .set_index("variable_name")["derive13_np"]
    .to_dict()
)
dict_ncs51: dict[str, str] = (
    dictionary.dropna(subset=["derive_ncs51_np"])
    .set_index("variable_name")["derive_ncs51_np"]
    .to_dict()
)

In [ ]:
# Filter data based on columns and samples
df_excluded: pd.DataFrame = derive54.loc[
    derive54["SUBJECTID"].isin(id_excluded), list(dict_derive54.values())
].reset_index(drop=True)
df_excluded: pd.DataFrame = df_excluded.join(
    derive13.set_index("SUBJECTID")[dict_derive13.values()], on="SUBJECTID", how="inner"
).join(ncs51.set_index("SUBJECTID")[dict_ncs51.values()], on="SUBJECTID", how="inner")

In [12]:
# Label stage as included or excluded
stage_list: list[str] = ["included", "excluded"]

In [13]:
# Label excluded samples
df_excluded["stage"] = stage_list[1]

In [ ]:
# For cholesterols and triglycerides, convert from SI unit mM to mg/dL
for _, row in dictionary.dropna(subset="conversion_factor").iterrows():
    variable_name: str = row.loc["derive54_np"]
    df_excluded[variable_name] = (
        df_excluded[variable_name] / row["conversion_factor"]
    ).astype(float)

### Concatenate existing pilot data with excluded population data

In [ ]:
# Use the existing demographic data as the included population
df_included: pd.DataFrame = pilot.loc[
    pilot["SubjectID"].isin(id_included)
].convert_dtypes()

In [ ]:
# Rename pilot columns to their original names in ARIC variable dictionary
df_included: pd.DataFrame = (
    df_included.rename(columns=dict_derive54)
    .rename(columns=dict_derive13)
    .rename(columns=dict_ncs51)
)

In [17]:
# Concatenate included and excluded population, if the column is not in df_excluded, fill with NaN
df_included["stage"] = stage_list[0]
df_concat = pd.concat([df_included, df_excluded], axis=0, ignore_index=True)

### Join additional brain MRI data

In [ ]:
# Specify columns and visit code to use for MRI data
usecols_mri: list[str] = [
    "EstimatedTotalIntraCranialVol",
    "temporal_parietal_metaroi_vol",
]
mri_v5: pd.DataFrame = mri.loc[mri["VISIT"].isin(["V5"])].copy()
mri_v7: pd.DataFrame = mri.loc[mri["VISIT"].isin(["V6V7"])].copy()
mri_v11: pd.DataFrame = mri.loc[mri["VISIT"].isin(["V8V11"])].copy()

In [ ]:
# Join MRI data to the concatenated data
df: pd.DataFrame = (
    df_concat.join(
        mri_v5.set_index("SUBJECTID")[usecols_mri], on="SUBJECTID", how="left"
    )
    .join(
        mri_v7.set_index("SUBJECTID")[usecols_mri],
        on="SUBJECTID",
        how="left",
        rsuffix="_V6V7",
    )
    .join(
        mri_v11.set_index("SUBJECTID")[usecols_mri],
        on="SUBJECTID",
        how="left",
        rsuffix="_V8V11",
    )
)

In [ ]:
# Sanitize categorical variables
df["stage"] = pd.Categorical(df["stage"], categories=stage_list, ordered=True)
df["CURSMK52"] = pd.Categorical(df["CURSMK52"], categories=[0, 1], ordered=True)
df["GENDER51"] = pd.Categorical(df["GENDER51"], categories=["F", "M"], ordered=True)
df["RACEGRP51"] = pd.Categorical(
    df["RACEGRP51"], categories=["A", "B", "I", "W"], ordered=True
)
df["CENTER"] = pd.Categorical(
    df["CENTER"], categories=["F", "J", "M", "W"], ordered=True
)
df["DIABTS54"] = pd.Categorical(df["DIABTS54"], categories=[0, 1], ordered=True)
df["HYPERT55"] = pd.Categorical(df["HYPERT55"], categories=[0, 1], ordered=True)
df["CHOLMDCODE53"] = pd.Categorical(df["CHOLMDCODE53"], categories=[0, 1], ordered=True)
df["HYPTMDCODE52"] = pd.Categorical(df["HYPTMDCODE52"], categories=[0, 1], ordered=True)
df["STATINCODE52"] = pd.Categorical(df["STATINCODE52"], categories=[0, 1], ordered=True)
df["PRVCHD51"] = pd.Categorical(df["PRVCHD51"], categories=[0, 1], ordered=True)
df["COGDIAG51"] = pd.Categorical(
    df["COGDIAG51"], categories=["N", "U", "M", "D"], ordered=True
)
df["ELEVEL02"] = pd.Categorical(df["ELEVEL02"], categories=[1, 2, 3], ordered=True)
df["PREVDEFPOSSHF51"] = pd.Categorical(
    df["PREVDEFPOSSHF51"], categories=[0, 1], ordered=True
)
df["PRVSTR51"] = pd.Categorical(df["PRVSTR51"], categories=[0, 1], ordered=True)

In [21]:
# Handle missing values
df["globalcortex"] = df["globalcortex"].fillna(0)
df["FINAL_APOE"] = df["FINAL_APOE"].fillna(0)
df["FINAL_APOE"] = df["FINAL_APOE"].astype(str).str.contains("4").astype(int)

In [22]:
# Compute demographic characteristics
df_stat = demographic_characteristics(df, stage_list)

In [23]:
# Save demographic characteristics to CSV
df_stat.to_csv(path_output_demographic_characteristics, index=True, encoding="utf-8")

### Join lipoprotein data

In [ ]:
# Define the list of lipoproteins
lipoprotein_list: list[str] = lipoprotein.columns[4:129].tolist()
df_lipoprotein_list: pd.DataFrame = pd.DataFrame(
    lipoprotein_list, columns=["lipoprotein"]
)

In [ ]:
# Subset for included samples
df_demographics: pd.DataFrame = df.loc[df["stage"] == "included"].drop(
    columns=["stage"]
)

In [ ]:
# Join lipoprotein data
df_merge: pd.DataFrame = df_demographics.join(
    lipoprotein.set_index("ID")[lipoprotein_list], on="SUBJECTID", how="inner"
)

### Output

In [27]:
# Output
df_lipoprotein_list.to_csv(path_output_lipoprotein_list, index=False)
df_merge.to_csv(path_output_pilot, index=False)